In [ ]:
from tf_utils import TFMixin

In [ ]:
# config
tensorboard_logdir = "./data/experiments/t5/logs"
checkpoint_savedir = "./data/experiments/t5/models"

# hyperparams
batch_size = 4
num_parallel = 2
epochs = 1
model_arch = "SnapthatT5"
model_src = "t5-small"
learning_rate = 1e-6

In [ ]:
# Although it doesn't directly solve your problem with TQDM, 
    # multiple inheritance is a convenient pattern for Metaflow generally that makes these issues far less painful.
# This TFMixin thing is a parent of the FlowSpec child class.
tf_handler = TFMixin()

In [ ]:
# The reason to do this is to do iterate quickly in the NB with all the progress bars and iterative Jupyter feel, 
    # then once the code runs as you want it is a simple copy-pasta to port over to the FlowSpec and harden the workflow.
    # Wherever `tf_handler.` is used in this cell, replace with `self.` in the FlowSpec tasks.
    # The other minor thing to do during the copy-pasta is use self. in front of hyperparams in the FlowSpec, 
        # since they can be set as Metaflow params which you can dynamically set at the flow runtime. 
    # See flow_single.py for an example that is implemented.

tf_train_ds, tf_valid_ds, train_steps, valid_steps = tf_handler.get_data(
    'squad', pct='10', batch_size = batch_size, num_workers=1)

callbacks = tf_handler.configure_callbacks(tensorboard_logdir, checkpoint_savedir)

tuning_set = {'learning_rate': learning_rate}
model = tf_handler.compile_model(model_src = model_src, hyperparams=tuning_set)

In [ ]:
# Let keras do the magic, with progress bars! 🎉 🥳
model.fit(
    tf_train_ds,
    epochs=1,
    steps_per_epoch=train_steps,
    validation_data=tf_valid_ds, 
    validation_steps=valid_steps,
    callbacks=callbacks, 
)